# Heroes Of Pymoli Data Analysis

* Of the 576 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 with secondary groups falling between 15-19 and 25-29. Within the peak demographic group most purchases are made by 20 year olds at 12.0% of purchases, followed by 23 and 22 at 8.5%

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "Resources/purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Player Count

In [2]:
# Calculate the number of total players in the DataFrame
total_players = len(purchase_data["SN"].unique())

# Place all of the data found into a count DataFrame
count_table = pd.DataFrame({"Total Players": [total_players]})
count_table

,Total Players
0,576


# Purchasing Analysis (Total)

In [3]:
# Calculate the number of unique items in the DataFrame
num_items = len(purchase_data["Item ID"].unique())


# Calculate the average item price
avg_price = purchase_data["Price"].mean()


# Calculate the number of purchases
num_purchases = purchase_data["Purchase ID"].count()


# Calculate the total revenue
total_revenue = purchase_data["Price"].sum()

# Place all of the data found into a summary DataFrame
summary_table = pd.DataFrame({"Number of Unique Items": [num_items],
                             "Average Price": [avg_price],
                             "Number of Purchases": [num_purchases],
                             "Total Revenue": [total_revenue]
                             })
# Use Map to format money columns
summary_table["Average Price"] = summary_table["Average Price"].map("${:.2f}".format)
summary_table["Total Revenue"] = summary_table["Total Revenue"].map("${:.2f}".format)

summary_table

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$3.05,780,183,$2379.77


# Gender Demographics

In [4]:
# create a list of non duplicate screen names
no_dup_sn = purchase_data.drop_duplicates(["SN"], keep ='last')

# count gender
count_gender = no_dup_sn["Gender"].value_counts().reset_index()
count_gender["Persentage of Players"] = count_gender["Gender"] / total_players * 100
count_gender.rename(columns = {"index": "Gender", "Gender": "Total Count"}, inplace = True)
count_gender["Persentage of Players"] = count_gender["Persentage of Players"].map("{:.2f}%".format)
count_gender



,Gender,Total Count,Persentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%


# Purchasing Analysis (Gender)

In [5]:
# counts purchases by gender
pur_ct_gen = pd.DataFrame(purchase_data.groupby("Gender")["Gender"].count().reset_index(name = "Purchase Count"))

# total purchases by gender
pur_tot_gen = pd.DataFrame(purchase_data.groupby("Gender")["Price"].sum().reset_index(name = "Total Purchase Value"))

# avg purchase price by gender
pur_avg_gen = pd.DataFrame(purchase_data.groupby("Gender")["Price"].mean().reset_index(name = "Average Purchase Price"))

# merge first two tables
merge_table1 = pd.merge(pur_ct_gen, pur_tot_gen , on="Gender")
merge_table1["Normalized Totals"] = merge_table1["Total Purchase Value"] / merge_table1["Purchase Count"]

# merge last table
merge_table = pd.merge(merge_table1, pur_avg_gen , on="Gender")
# format
merge_table["Total Purchase Value"] = merge_table["Total Purchase Value"].map("${:.2f}".format)
merge_table["Average Purchase Price"] = merge_table["Average Purchase Price"].map("${:.2f}".format)
merge_table["Normalized Totals"] = merge_table["Normalized Totals"].map("${:.2f}".format)

merge_table


,Gender,Purchase Count,Total Purchase Value,Normalized Totals,Average Purchase Price
0,Female,113,$361.94,$3.20,$3.20
1,Male,652,$1967.64,$3.02,$3.02
2,Other / Non-Disclosed,15,$50.19,$3.35,$3.35


# Purchasing Analysis (Age)

In [6]:
# get data for age groups
age_groups= pd.DataFrame(purchase_data)

# Establish bins for ages
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

age_groups["View Group"] = pd.cut(age_groups["Age"], age_bins, labels=group_names)

# Creating a group based off of the bins

# Purchase Count by age
age_purchases = pd.DataFrame(age_groups.groupby("View Group")["Purchase ID"].count().reset_index(name = "Purchase Count"))


# Average purchase price by age
age_avg_pur = pd.DataFrame(age_groups.groupby("View Group")["Price"].mean().reset_index(name = "Average Purchase Price"))


# Total Purchase Value by age
age_total_val = pd.DataFrame(age_groups.groupby("View Group")["Price"].sum().reset_index(name = "Total Purchase Value"))


# merge first two tables
merge_age_tbl1 = pd.merge(age_purchases, age_total_val , on="View Group")
merge_age_tbl1["Normalized Totals"] = merge_age_tbl1["Total Purchase Value"] / merge_age_tbl1["Purchase Count"]

# merge last table
merge_age_tbl = pd.merge(merge_age_tbl1, age_avg_pur , on="View Group")

# format
merge_age_tbl["Total Purchase Value"] = merge_age_tbl["Total Purchase Value"].map("${:.2f}".format)
merge_age_tbl["Average Purchase Price"] = merge_age_tbl["Average Purchase Price"].map("${:.2f}".format)
merge_age_tbl["Normalized Totals"] = merge_age_tbl["Normalized Totals"].map("${:.2f}".format)

merge_age_tbl


,View Group,Purchase Count,Total Purchase Value,Normalized Totals,Average Purchase Price
0,<10,23,$77.13,$3.35,$3.35
1,10-14,28,$82.78,$2.96,$2.96
2,15-19,136,$412.89,$3.04,$3.04
3,20-24,365,$1114.06,$3.05,$3.05
4,25-29,101,$293.00,$2.90,$2.90
5,30-34,73,$214.00,$2.93,$2.93
6,35-39,41,$147.67,$3.60,$3.60
7,40+,13,$38.24,$2.94,$2.94


## Top Spenders

In [7]:
#Purchase Count by screen name
sn_purchases = pd.DataFrame(purchase_data.groupby("SN")["Purchase ID"].count().reset_index(name = "Purchase Count"))


#Average purchase price by screen name
sn_avg_pur = pd.DataFrame(purchase_data.groupby("SN")["Price"].mean().reset_index(name = "Average Purchase Price"))


# Total Purchase Value by screen name
sn_total_val = pd.DataFrame(purchase_data.groupby("SN")["Price"].sum().reset_index(name = "Total Purchase Value"))

# merge first two tables
merge_sn_tbl1 = pd.merge(sn_purchases, sn_avg_pur , on="SN")

# merge last table
merge_sn_tbl = pd.merge(merge_sn_tbl1, sn_total_val , on="SN")
merge_sn_tbl = merge_sn_tbl.sort_values("Total Purchase Value", ascending=False)

# format
merge_sn_tbl["Total Purchase Value"] = merge_sn_tbl["Total Purchase Value"].map("${:.2f}".format)
merge_sn_tbl["Average Purchase Price"] = merge_sn_tbl["Average Purchase Price"].map("${:.2f}".format)

# print top 5 
merge_sn_tbl.head(5)


,SN,Purchase Count,Average Purchase Price,Total Purchase Value
360,Lisosia93,5,$3.79,$18.96
246,Idastidru52,4,$3.86,$15.45
106,Chamjask73,3,$4.61,$13.83
275,Iral74,4,$3.40,$13.62
281,Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [8]:
# Create items table
items_table = purchase_data[["Item ID", "Item Name", "Price"]]

# Create the GroupBy object based on the "Item ID" and "Item Name" column
items_group = pd.DataFrame(items_table.groupby(["Item ID", "Item Name", "Price"])["Item ID"].count().reset_index(name = "Purchase Count"))
items_group["Total Purchase Value"] = items_group["Purchase Count"] * items_group["Price"]

# Sort Items from high to low
items_sort = items_group.sort_values("Purchase Count", ascending=False)

# format
items_sort["Total Purchase Value"] = items_sort["Total Purchase Value"].map("${:.2f}".format)
items_sort["Price"] = items_sort["Price"].map("${:.2f}".format)

# Reorganizing the columns using double brackets
items_sort = items_sort[["Item ID", "Item Name", "Purchase Count", "Price","Total Purchase Value"]]


items_sort.head(5)


,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
177,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
144,145,Fiery Glass Crusader,9,$4.58,$41.22
107,108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
81,82,Nirvana,9,$4.90,$44.10
19,19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [9]:
# Sort Items from high to low
items_sort_profit = items_group.sort_values("Total Purchase Value", ascending=False)

# format
items_sort_profit["Total Purchase Value"] = items_sort_profit["Total Purchase Value"].map("${:.2f}".format)
items_sort_profit["Price"] = items_sort_profit["Price"].map("${:.2f}".format)

# Reorganizing the columns using double brackets
items_sort_profit = items_sort_profit[["Item ID", "Item Name", "Purchase Count", "Price","Total Purchase Value"]]


items_sort_profit.head(5)


,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
177,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
81,82,Nirvana,9,$4.90,$44.10
144,145,Fiery Glass Crusader,9,$4.58,$41.22
91,92,Final Critic,8,$4.88,$39.04
102,103,Singed Scalpel,8,$4.35,$34.80
